<a href="https://colab.research.google.com/github/NitCoh/TextAnalysisForBigData/blob/master/Big_Data_Analysis_for_Cyber_Security_Text_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import re
import pandas as pd
import numpy as np

DATA_COLS:
Columns:
FileNameID,Sha256ID,Md5ID,Sha1ID,IssuerID,SignerID,PublisherID,WebFileIpID,WebFileUrlID,ThreatFamilyID,ThreatNameID,SigSeqID,SignatureNameID,MachineGuidID,RowNumber,PeAttributesString,WebFileUrlDomain,WebFileUrlHostname,Ctph,LsHash,ThreatCategory,ReportTime,SpynetReportGuid,Firewall,Size,Version,Type,SignerHash,IssuerHash,IsValidCertificate,CertInvalidDetails,Checksum,IsRuntimePacked,FileAccess,Screensaver,StartupUserFolder,MarkOfTheWeb,UsnTimestamp,IsInContainer,IsContainer,IsPEFile,IsAutoRunInf,PeTimestamp

Upload the data file!

In [0]:
COLS = "FileNameID,Sha256ID,Md5ID,Sha1ID,IssuerID,SignerID,PublisherID,WebFileIpID,WebFileUrlID,ThreatFamilyID,ThreatNameID,SigSeqID,SignatureNameID,MachineGuidID,RowNumber,PeAttributesString,WebFileUrlDomain,WebFileUrlHostname,Ctph,LsHash,ThreatCategory,ReportTime,SpynetReportGuid,Firewall,Size,Version,Type,SignerHash,IssuerHash,IsValidCertificate,CertInvalidDetails,Checksum,IsRuntimePacked,FileAccess,Screensaver,StartupUserFolder,MarkOfTheWeb,UsnTimestamp,IsInContainer,IsContainer,IsPEFile,IsAutoRunInf,PeTimestamp"
cols_names = COLS.split(",")
cols_names

In [17]:
## Upload the data file first!
data_path = '/content/Obf_oneInTenWeek1_d1.tsv'

df = pd.read_csv(data_path,sep='\t', names=cols_names)
df.head()

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (39,40,41) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,FileNameID,Sha256ID,Md5ID,Sha1ID,IssuerID,SignerID,PublisherID,WebFileIpID,WebFileUrlID,ThreatFamilyID,ThreatNameID,SigSeqID,SignatureNameID,MachineGuidID,RowNumber,PeAttributesString,WebFileUrlDomain,WebFileUrlHostname,Ctph,LsHash,ThreatCategory,ReportTime,SpynetReportGuid,Firewall,Size,Version,Type,SignerHash,IssuerHash,IsValidCertificate,CertInvalidDetails,Checksum,IsRuntimePacked,FileAccess,Screensaver,StartupUserFolder,MarkOfTheWeb,UsnTimestamp,IsInContainer,IsContainer,IsPEFile,IsAutoRunInf,PeTimestamp
0,7689280,17046923,10489946,9382543,9641,46623,188146,462213,5905091,2720,5644,1036378,227,18220780,14467170,NaN,mediafire.com,download2089.mediafire.com,98304:H5y4BLvb5sTOKPg6JqJTkIMMUaVlebVwaccuyPNA...,77e66aabaf5adeaf55b5adb95f5bdfbdb6bf66eefd7d99...,NaN,1/1/2017 7:18:01 PM,528e69bc-405f-4713-a1f6-36c1903116d0,NaN,7041700,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,NaN,NaN,internet,NaN,NaN,NaN,False,NaN,NaN
1,7689292,1024138,18599752,17052884,9641,46623,188146,100848,15003067,2720,5644,24632,616,20983989,16606030,"1,5,21,23,45,84,85,89,94,98,106,109,110,113,11...",mpgh.net,www.mpgh.net,3072:Gnw+4gXWKP0XzBwWZzqZBcsmhVnxC3eClBFydr:6J...,b7bafa5fa7a5d5ee77bee55f6765e55faee6e96b5f95f5...,NaN,1/1/2017 10:22:19 PM,fd4ef558-8f17-4366-9d5f-abc625582d7f,NaN,101888,NaN,NaN,NaN,NaN,NaN,NaN,0001fb8f,0.0,2,NaN,NaN,internet,1/1/2017 11:22:22 PM,True,NaN,True,NaN,3/6/2016 3:43:13 AM
2,14661295,3242391,4079241,7856388,9641,46623,188146,570877,4648460,2720,5644,1036485,632,26901046,21310830,"1,5,21,23,25,31,45,63,73,94,106,109,110,112,11...",pcinspector.de,download.pcinspector.de,12288:FfOSr3qnux6sHMfQFbN7RaWUH56yRebvnIJ94U22...,77e6da7f57bd9dbafdeea65bb7559d6efd56eba76e5555...,NaN,1/1/2017 1:14:55 PM,a371233e-a97c-4a6f-b032-a53403a949ac,NaN,767771,6.10.0.6100,APP,NaN,NaN,NaN,NaN,00000000,1.0,2,NaN,NaN,unknown,1/1/2017 1:14:39 PM,NaN,NaN,True,NaN,9/23/2002 11:55:47 AM
3,7689292,1024138,18599752,17052884,9641,46623,188146,100848,15003067,2720,5644,24632,616,9649625,7582810,"1,5,21,23,45,84,85,89,94,98,106,109,110,113,11...",mpgh.net,www.mpgh.net,3072:Gnw+4gXWKP0XzBwWZzqZBcsmhVnxC3eClBFydr:6J...,b7bafa5fa7a5d5ee77bee55f6765e55faee6e96b5f95f5...,NaN,1/1/2017 11:06:32 PM,65605608-5145-4c86-b76c-573d1b56819a,True,101888,NaN,NaN,NaN,NaN,NaN,NaN,0001fb8f,0.0,2,NaN,NaN,internet,NaN,True,NaN,True,NaN,3/6/2016 3:43:13 AM
4,7689305,1012018,8180486,14860251,9641,46623,188146,955856,2641005,2720,5644,1036641,221,21896518,17330640,NaN,forumprofi.de,clan-z.forumprofi.de,196608:skgNapTLYtSXzUaUykAmSs5iMKrbgdA50eQEvZP...,796a9fb765fbaea5e59e6ff56ea9be67b7a95afd9fe6b5...,NaN,1/1/2017 9:51:00 AM,3c5719d9-8fc9-46c1-b874-c5803cb93b13,True,8254731,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN,internet,NaN,NaN,NaN,False,NaN,NaN
